# Excipient Compatibility Data Extraction and Curation

*This notebook is meant for the analysis of excipient compatibility data previously done with BioAssay
It consists of multiple steps:*
1. Upload of .csv files as generated by Empower
2. Reformatting of the data for analysis using dataframes
3. Calculation of relative retention times and binning of retention times
4. Attribution of retention time shifts and probabilty assignment

## Upload of Degradation Data (Week 0-5, wet/dry)

In [1]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np


C:\Users\dejejord\AppData\Local\Temp\ipykernel_24336\3342246258.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd #handling of csv files and dataframes


In [18]:
#This part was an attempt of having an interface to click and select files 
#It does currently not work

from tk import  Checkbutton, Button, IntVar

def get_selected_items():
    global csv_file_path
    selected_items = [item.get() for item in checkbox_vars]
    csv_file_path = []
    print("Selected items:", selected_items)
    for j,counter in enumerate(selected_items):
        if counter==True:
            csv_file_path.append(filedialog.askopenfilename(title="Select CSV File for ", filetypes=[("CSV Files", "*.csv")]))
            # Display the selected file path
            print("Selected CSV File for ",checkbox_names[j], " is: ", csv_file_path[j])
        else: csv_file_path.append("0")
    root.quit()

# Create the Tkinter root window
root = Tk()

# Create tkinter IntVar variables to store the state of the checkboxes
checkbox_vars = [IntVar() for _ in range(6)]
checkbox_names = ["Week0","Week0_humid","Week2","Week2_humid","Week5","Week5_humid"]

# Create the checkboxes
checkboxes = []
for i, var in enumerate(checkbox_vars):
    checkbox = Checkbutton(root, text=checkbox_names[i], variable=var)
    checkbox.pack()
    checkboxes.append(checkbox)

# Create a button to get the selected items
button = Button(root, text="Get Selected Items", command=get_selected_items)
button.pack()

# Run the Tkinter event loop
root.mainloop()

TypeError: 'module' object is not callable

In [2]:
#Loading of xlsx-files and reformatting of dataframe - Week2 loaded as example
df = pd.read_excel("C:/Users/dejejord/OneDrive - Merck Sharp & Dohme LLC/Desktop/ExcipientCompatibility_Jupyter/Week2_Trimmed.xlsx")

#Trim down to most important columns
df1 = df[["Vial","Retention Time", "Area"]]

#Get relative retention times by division by retention time of largest area peak ==> Caveat: largest area needs to be API peak
max_area = df1["Area"].max()
max_area_row=df1[df1["Area"] == max_area]
max_Ret_value = max_area_row["Retention Time"].iloc[0]
df1["Retention Time"] = df1["Retention Time"] / max_Ret_value

#Pivot dataframe to be sorted by retention times
df1=df1.round(2)
pivot_df1 = df1.pivot_table(index="Vial", columns="Retention Time", values="Area",aggfunc='sum')
pivot_df1[np.isnan(pivot_df1)] = 0

#Merge relative retention times if difference is 0.01 and no entry recorded for one of two columns for each well
df_merged = pd.DataFrame()
col_del = []
columns = pivot_df1.columns.tolist()
c=0

while c <= len(columns)-2:
    merge_condition = False
    for index,row in pivot_df1.iterrows():
        if round((columns[c+1]-columns[c]),2)==0.01:
            if (row[columns[c+1]] != 0.0 and row[columns[c]] == 0.0) or (row[columns[c+1]] == 0.0 and row[columns[c]] != 0.0):
                merge_condition = True
                df_merged[columns[c]] =  pivot_df1[columns[c]]+pivot_df1[columns[c+1]]
                c=c+2
                break
    if merge_condition == False:
        df_merged[columns[c]] =  pivot_df1[columns[c]]
        c=c+1
if c == len(columns)-1:
    df_merged[columns[c]] =  pivot_df1[columns[c]]


#Normalize by API peak area
final_norm = df_merged.div(df_merged[df_merged.columns[df_merged.columns == 0.99]].values,axis=0)
final_norm=final_norm.round(5)

C:\Users\dejejord\AppData\Local\Temp\ipykernel_24336\100699593.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Retention Time"] = df1["Retention Time"] / max_Ret_value


In [259]:
df.set_index(["Vial"])["SampleName"]

Vial
1:A,1         2Week_40C
1:A,1         2Week_40C
1:A,1         2Week_40C
1:A,10        2Week_40C
1:A,10        2Week_40C
              ...      
2:F,7     2Week_40C75RH
2:F,8     2Week_40C75RH
2:F,8     2Week_40C75RH
2:F,9     2Week_40C75RH
2:F,9     2Week_40C75RH
Name: SampleName, Length: 312, dtype: object

In [3]:
non_zero_counts = np.count_nonzero(df_merged, axis=0)
result = pd.Series(non_zero_counts, index=df_merged.columns)

print(result)

0.11      1
0.32      5
0.34      1
0.99    138
1.01      5
1.11    137
1.17     16
1.41      4
1.53      1
dtype: int64


In [4]:
final_norm

,0.11,0.32,0.34,0.99,1.01,1.11,1.17,1.41,1.53
Vial,,,,,,,,,
"1:A,1",0.0,0.0,0.0,1.0,0.0,0.00383,0.0,0.04941,0.0
"1:A,10",0.0,0.0,0.0,1.0,0.0,0.00390,0.0,0.00000,0.0
"1:A,11",0.0,0.0,0.0,1.0,0.0,0.00382,0.0,0.00000,0.0
"1:A,12",0.0,0.0,0.0,1.0,0.0,0.00386,0.0,0.00000,0.0
"1:A,2",0.0,0.0,0.0,1.0,0.0,0.00387,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...
"2:F,5",0.0,0.0,0.0,1.0,0.0,0.00389,0.0,0.00000,0.0
"2:F,6",0.0,0.0,0.0,1.0,0.0,0.00376,0.0,0.00000,0.0
"2:F,7",0.0,0.0,0.0,1.0,0.0,0.00388,0.0,0.00000,0.0
